## knock 071 2つの日付から経過月数を計算する

In [7]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列の表示が省略されないようにする
import polars.selectors as cs# 抽出条件のプリセット

### データを読み込む

In [8]:
df_customer = pl.read_csv("../docker/work/data/customer.csv")
display(df_customer.head())

df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
display(df_receipt.head())

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,i64,str,str,i64,str,str,str,i64,str
"""CS021313000114…","""大野 あや子""",1,"""女性""","""1981-04-29""",37,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20150905,"""0-00000000-0"""
"""CS037613000071…","""六角 雅彦""",9,"""不明""","""1952-04-01""",66,"""136-0076""","""東京都江東区南砂******…","""S13037""",20150414,"""0-00000000-0"""
"""CS031415000172…","""宇多田 貴美子""",1,"""女性""","""1976-10-04""",42,"""151-0053""","""東京都渋谷区代々木*****…","""S13031""",20150529,"""D-20100325-C"""
"""CS028811000001…","""堀井 かおり""",1,"""女性""","""1933-03-27""",86,"""245-0016""","""神奈川県横浜市泉区和泉町**…","""S14028""",20160115,"""0-00000000-0"""
"""CS001215000145…","""田崎 美紀""",1,"""女性""","""1995-03-29""",24,"""144-0055""","""東京都大田区仲六郷*****…","""S13001""",20170605,"""6-20090929-2"""


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


### ノック

In [9]:
(
    df_receipt
    # 使用する列を抽出する(大規模データの場合メモリ使用量を節約する効果がある)
    .select(
        pl.col("customer_id"),
        pl.col("sales_ymd")
    )
    # 同一売上日かつ同一顧客のデータを削除する
    .unique()
    # 顧客IDに会員登録日を追加する
    .join(
        other = df_customer[["customer_id", "application_date"]],
        on = "customer_id",
        how = "left"
    )
    # DateTime型に変換する
    .with_columns([
        pl.col("sales_ymd").cast(pl.Utf8).str.strptime(dtype = pl.Date, format = "%Y%m%d"),
        pl.col("application_date").cast(pl.Utf8).str.strptime(dtype = pl.Date, format = "%Y%m%d")
    ])
    # 入会日から買い物までの経過月数を計算する
    .with_columns(
        ((pl.col("sales_ymd").dt.year() - pl.col("application_date").dt.year()) * 12
        + (pl.col("sales_ymd").dt.month() - pl.col("application_date").dt.month()))
        .alias("elapsed_months")
    )
    # 先頭10行
    .head(n = 10)
)

customer_id,sales_ymd,application_date,elapsed_months
str,date,date,i64
"""CS025415000050…",2018-08-21,2016-01-31,31
"""CS003515000195…",2019-06-05,2015-03-06,51
"""CS039414000052…",2018-05-06,2015-05-09,36
"""ZZ000000000000…",2019-01-05,null,null
"""CS026615000085…",2018-11-01,2015-08-02,39
"""CS028415000203…",2017-05-23,2015-02-04,27
"""CS014214000013…",2019-04-28,2015-03-05,49
"""CS034414000034…",2019-08-19,2015-05-16,51
"""CS026414000062…",2019-06-19,2015-06-10,48
